# Object Recognition: YoloV8

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [ ]:
!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.4.0+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.8/107.7 GB disk)


In [ ]:
!pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 kB 17.3 MB/s eta 0:00:00


In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

!pip install roboflow --quiet

from roboflow import Roboflow
rf = Roboflow(api_key="Gl773kBPkL1O8pZYCp2y")
project = rf.workspace("cs473-jxlk7").project("cs473-sbytd")
version = project.version(3)
dataset = version.download("yolov8")


/content/datasets
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.4 MB/s eta 0:00:00
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to CS473-3 in yolov8:: 100%|██████████| 275/275 [00:00<00:00, 4997.24it/s]


In [ ]:
%cd {HOME}

!yolo task=detect mode=train model=yolov8s.pt data={dataset.location}/data.yaml epochs=50 imgsz=800 plots=True

/content
/usr/local/lib/python3.10/dist-packages/ultralytics/nn/tasks.py:567: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_location='cpu'), file

In [ ]:
%cd {HOME}

from ultralytics import YOLO
import os
import easyocr
import cv2
from PIL import Image

from google.colab import drive
drive.mount('/content/drive')

# Load YOLO model and EasyOCR
model = YOLO('runs/detect/train5/weights/best.pt')
reader = easyocr.Reader(['en'])

# Path to images
images_path = '/content/drive/My Drive/CS473/OD_OCR_testing/images'

# Create output directory if it doesn't exist
output_dir = f"{HOME}/text"
os.makedirs(output_dir, exist_ok=True)

# Get list of image files
image_files = [f for f in os.listdir(images_path) if f.endswith(('.png', '.jpg', '.jpeg'))] # Filter out non-image files
print(image_files)

# Loop through images
for image_file in image_files:
    image_path = f'{images_path}/{image_file}'
    image = cv2.imread(image_path)
    results = model.predict(source=image_path, conf=0.5, iou=0.4, save=True)

    # Initialize text output for this image
    output_text = ""

    for result_idx, result in enumerate(results):
        boxes = result.boxes  # Get bounding boxes
        class_ids = boxes.cls  # Class IDs
        xyxy = boxes.xyxy  # Bounding box coordinates

        for i, box in enumerate(boxes):
            class_id = int(class_ids[i])  # Convert to integer for the class ID
            # Here you should map the class ID to the class name, assuming you have a list of class names
            class_name = model.names[class_id]  # Assuming YOLO model has class names

            # Prepare output format with the class name as the first word
            output_text += f"['{class_name}', "

            x1, y1, x2, y2 = [int(coord) for coord in xyxy[i]]

            # Crop the detected region from the image
            cropped_image = image[y1:y2, x1:x2]

            # Convert the cropped image to RGB (if necessary)
            cropped_image_rgb = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB)

            # Perform OCR on the cropped region
            ocr_result = reader.readtext(cropped_image_rgb)

            # Add OCR results to output text in the specified format
            if ocr_result:
                for ocr_data in ocr_result:
                    text = ocr_data[1]  # Detected text
                    output_text += f"'{text}', "

                # Replace last comma with closing bracket
                output_text = output_text.rstrip(', ') + "]\n"
            else:
                # If no text detected, close the bracket
                output_text = output_text.rstrip(', ') + "no text found]\n"

    # Extract the base name of the image (without extension)
    image_name = os.path.splitext(image_file)[0]

    # Save the results to a .txt file with the same base name as the image in the {HOME}/text folder
    with open(f'{output_dir}/{image_name}.txt', 'w') as f:
        f.write(output_text)

    print(f"OCR results saved to {output_dir}/{image_name}.txt")

/content
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['1.png', '2.png', '4.png', '3.png', '7.png', '8.png', '5.png', '6.png']



image 1/1 /content/drive/My Drive/CS473/OD_OCR_testing/images/1.png: 256x800 3 entitys, 2 rels, 7 rel_attrs, 12.8ms
Speed: 3.1ms preprocess, 12.8ms inference, 2.4ms postprocess per image at shape (1, 3, 256, 800)
Results saved to runs/detect/predict7



OCR results saved to /content/text/1.txt


image 1/1 /content/drive/My Drive/CS473/OD_OCR_testing/images/2.png: 736x800 3 entitys, 1 ident_rel, 4 rels, 1 weak_entity, 12.9ms
Speed: 4.8ms preprocess, 12.9ms inference, 1.5ms postprocess per image at shape (1, 3, 736, 800)
Results saved to runs/detect/predict7



OCR results saved to /content/text/2.txt


...

OCR results saved to /content/text/8.txt


image 1/1 /content/drive/My Drive/CS473/OD_OCR_testing/images/5.png: 800x800 3 entitys, 2 ident_rels, 3 rels, 2 weak_entitys, 12.0ms
Speed: 4.9ms preprocess, 12.0ms inference, 1.5ms postprocess per image at shape (1, 3, 800, 800)
Results saved to runs/detect/predict7


OCR results saved to /content/text/5.txt



image 1/1 /content/drive/My Drive/CS473/OD_OCR_testing/images/6.png: 416x800 5 entitys, 5 rels, 10.1ms
Speed: 3.8ms preprocess, 10.1ms inference, 2.1ms postprocess per image at shape (1, 3, 416, 800)
Results saved to runs/detect/predict7


OCR results saved to /content/text/6.txt
